# Training
In this Notebook, we'll do the following.

* Install the `mmdetection` package
* Setup a configuration files to use our custom dataset for training and inference
* Visualize training progress
* Save our trained model to use for inference later
* Make inference with our trained model

### Requirements
* Linux or macOS (Windows is not currently officially supported)
* Python 3.6+
* PyTorch 1.3+
* CUDA 9.2+ (If you build PyTorch from source, CUDA 9.0 is also compatible)
* GCC 5+
* mmcv


### Install mmdetection 2.0
* Create a conda virtual environment and activate it.

In [2]:
#!conda create -n open-mmlab python=3.7 -y
#!conda activate open-mmlab

* Install PyTorch and torchvision (here we use a more stable torch and cuda version).

In [3]:
#!conda install pytorch=1.3.1 cudatoolkit=10.0 torchvision=0.4.2 -c pytorch

* Install pycocotools via the github repo instead of pypi because the pypi version is old and not compatible with the latest numpy.

In [5]:
# !git clone https://github.com/cocodataset/cocoapi.git
# !cd ~/cocoapi/PythonAPI
# !pip install cython
# !make install

* Clone the mmdetection repository.

In [6]:
# !git clone https://github.com/open-mmlab/mmdetection.git
# !cd mmdetection

* Install build requirements and then install mmdetection.

In [7]:
# !pip install -r requirements/build.txt
# !pip install -v -e .  # or !python setup.py develop

* (Optional: In case a rebuild is needed)

In [8]:
# !rm -rf build/*
# !python setup.py develop

### Setup Data Files
Add your customized COCO-formatted dataset <br /><br />
   * Add a new file, ‘mmdetection/mmdet/datasets/ade20kbed.py’.  CLASSES = None should be fine. <br /> <br /> 
   Or to be safe, list out your actual classes:
   ```CLASSES = ('door | doorframe, doorcase | double door', 'bed | bedpost | blanket, cover | headboard', 'wardrobe, closet, press', 'lamp', 'chest of drawers, chest, bureau, dresser | chest', 'pillow', 'armchair | chair | swivel chair', 'pane, pane of glass, window glass | windowpane, window', 'curtain, drape, drapery, mantle, pall', 'mirror', 'painting, picture | frame', 'wall', 'ceiling', 'floor, flooring', 'cabinet')```
   <br /><br />
   * We also provide the link to download this file here:
    <br /> <br /> (TODO) ```wget...```
    <br /> <br /> It will look like:
    ```
    from .coco import CocoDataset
    from .builder import DATASETS

    @DATASETS.register_module()
    class Ade20kBed(CocoDataset):
        CLASSES = (...) ```
  
* Then, add this dataset class to \_\_init\_\_.py
```
from .ade20kbed import Ade20kBed
...
__all__ = [...,'Ade20kBed']
     ```

### Modify config files
We will test out two different variations of models / backbone networks and provide instructions to modifying those files as well as links to download the config files.
   * Choosing a model to use:
    mmdetection's [Model Zoo](https://github.com/open-mmlab/mmdetection/blob/master/docs/model_zoo.md)
    <br />For example, if we want to use [Res2Net](https://github.com/open-mmlab/mmdetection/tree/master/configs/res2net) as backbone, and [Mask-RCNN](https://github.com/open-mmlab/mmdetection/tree/master/configs/mask_rcnn) as our model.
    <br />It is possible that the desired combination of model and backbone's performance information be listed under either the backbone page or the model page. Here, it's listed under the [backbone page](https://github.com/open-mmlab/mmdetection/tree/master/configs/res2net) for Res2Net.
    <br />We note the mask AP (38.7) and find the corresponding config file:
    ```configs/res2net/mask_rcnn_r2_101_fpn_2x_coco.py```
    
    
   * Then we can go into our mmdetection config directory and modify the corresponding files. In the newest mmdetection release, the config files use wrappers so they are more structured.
    <br />We look at the config file and figure out which other files we need to modify. Usually, we need to modify 2 files:
        * The base model config file
        * The dataset config file
   * Opening ```configs/res2net/mask_rcnn_r2_101_fpn_2x_coco.py``` 
   
   we see on the very top:
   
   ```_base_ = '../mask_rcnn/mask_rcnn_r50_fpn_2x_coco.py'```
   
    
   * Opening this we find:
    ```_base_ = [
    '../_base_/models/mask_rcnn_r50_fpn.py',
    '../_base_/datasets/coco_instance.py',
    '../_base_/schedules/schedule_2x.py', '../_base_/default_runtime.py'
]```
   * These are the files we should modify. Here we leave the schedule and runtime config files and only modify:
    ```mask_rcnn_r50_fpn.py``` and ```coco_detection.py``` (the base dataset config file for ```coco_instance.py```)
    
       * In ```mask_rcnn_r50_fpn.py```, we modify all ```num_classes``` and set them to 15, corresponding to the number of unique object classes in our dataset
       * In ```coco_instance.py```, we modify our ```data_root```, ```dataset_type```, and if needed, ```data = dict(...)``` where the files to our training/validation images and annotations are set.
        
   * We also provide the link to download pre-modified configuration files so you can replace the original config files directly.<br /><br />
    * (TODO: attach link to download directories and instructions to replace corresponding files)
    * (TODO: also modify schedule_***.py to change number of epochs for recording purpose)


### Training
Before we start training, make a working directory where your models will be saved. In your ```mmdetection``` root:
```
mkdir work_dirs
mkdir work_dirs/kdd/
mkdir work_dirs/kdd/mask_rcnn_res2net
```
Now we can finally start training!

```python tools/train.py configs/res2net/mask_rcnn_r2_101_fpn_2x_coco.py --work-dir work_dirs/kdd/mask_rcnn_res2net/```

### Visualizing traing Progress
```python tools/analyze_logs.py plot_curve work_dirs/kdd/mask_rcnn_res2net/[timestamp].log.json --keys segm_mAP segm_mAP_50 segm_mAP_75 --out work_dirs/kdd/mask_rcnn_res2net/loss_mask.png```
<img src="loss_plots/loss_mask.png">